<a href="https://colab.research.google.com/github/kenny101/Custom-Image-Classifier/blob/main/customClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create a Neural Network Image Classifier in Minutes!
For faster results, make sure your runtime is a GPU instance. To do this, simply:
  1. Click 'Runtime' 
  2. Select 'Change runtime type'
  3. Select 'GPU'
  4. Click 'Save'

Run steps 1-3. Note that more classes of image will require more time to download and train.


# Step 1: Installing Dependencies and Importing Modules

In [1]:
#Setup/Dependencies
!pip install fastbook
!pip install git+https://github.com/Joeclinton1/google-images-download.git


#import libraries
import fastbook
from fastbook import *
from fastai.vision.widgets import *
import urllib.request
import os
import IPython.display as display
import ipywidgets as widgets
from google_images_download import google_images_download


     |████████████████████████████████| 720 kB 8.0 MB/s 
     |████████████████████████████████| 188 kB 74.5 MB/s 
     |████████████████████████████████| 1.2 MB 31.6 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 51 kB 250 kB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
  Cloning https://github.com/Joeclinton1/google-images-download.git to /tmp/pip-req-build-5q2_qu_t
  Running command git clone -q https://github.com/Joeclinton1/google-images-download.git /tmp/pip-req-build-5q2_qu_t
     |████████████████████████████████| 904 kB 8.1 MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=15945 sha256=91cb7203801bd97107574730481b60d0f0c3fc9dfcb9404984471ad35ec9c511
  Stored in directory: /tmp/pip-ephem-wheel-c

# Step 2: User Input, Downloading the Data, and Training

In [4]:
print("How many things would you like to classify?")
num_classify = int(input())

classifiers = []
for i in range(1,num_classify+1):
  print(f"Type item {i}:")
  item = str(input())
  print("\n")
  classifiers.append(item)

#Downloading the Data
from google_images_download import google_images_download
response = google_images_download.googleimagesdownload()

for counter, my_class in enumerate(classifiers):
    arguments = {"keywords":my_class,
                 "limit":100,
                 "silent_mode":True,
                 "output_directory":"Datasets",
                 "image_directory":my_class,
                 "format":"jpg"}
    paths = response.download(arguments)

'''
Dataloader: Tells fastai:
(1) What kind of data do we have
(2) How to get a list of the images
(3) How to label the images
(4) How to create the validation set
'''
path = Path('Datasets')
myData = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))
dls = myData.dataloaders(path)

# Data Augmentation: changes the perspectives and scaling of the image
myData = myData.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = myData.dataloaders(path)

print("Training your model. Please wait.\n")
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

How many things would you like to classify?
5
Type item 1:
turtles


Type item 2:
dragons


Type item 3:
bears


Type item 4:
dinosaurs


Type item 5:
dogs




Unfortunately all 100 could not be downloaded because some images were not downloadable. 0 is all we got for this search filter!


Unfortunately all 100 could not be downloaded because some images were not downloadable. 97 is all we got for this search filter!


Unfortunately all 100 could not be downloaded because some images were not downloadable. 97 is all we got for this search filter!


Unfortunately all 100 could not be downloaded because some images were not downloadable. 0 is all we got for this search filter!


Unfortunately all 100 could not be downloaded because some images were not downloadable. 98 is all we got for this search filter!
Training your model. Please wait.



epoch,train_loss,valid_loss,error_rate,time
0,2.509113,1.239123,0.525641,00:14


epoch,train_loss,valid_loss,error_rate,time
0,1.107027,0.987487,0.333333,00:15
1,1.047753,1.147118,0.320513,00:15
2,0.912719,1.202170,0.333333,00:15
3,0.822054,1.192891,0.320513,00:14


# Step 3: Cleaning your dataset to produce more accurate results (Optional)
Keep, delete, or recategorize an image manually

In [3]:
cleaner = ImageClassifierCleaner(learn)
cleaner

# Step 4: Exporting your model and testing it on url image inputs

In [ ]:

#export the model
learn.export()
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner('export.pkl')

print('\n Model exported as: "export.pkl". Feel free to download it for future use. \n')

#Download a query to a folder
def downloadToDisplay(query, folder_name):

    #Download a single image of query
    arguments = {"keywords":query,
                 "limit":1,
                 "silent_mode":True,
                 "output_directory":folder_name,
                 "format":"jpg"}
    paths = response.download(arguments)
    
    #Rename file:
    src = paths[0][query][0]
    os.rename(src, f'{folder_name}/{query}/{query}.jpg')

#predict based on the image url input 
def predict_by_url(url):
  urllib.request.urlretrieve(url, "temp.jpg")
  pred,pred_idx,probs = learn_inf.predict("temp.jpg")
  print(f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}')

  downloadToDisplay(pred, "Images")

  #Display Image Results
  img1=open('temp.jpg','rb').read()
  wi1 = widgets.Image(value=img1, format='jpg', width=300, height=400)
  file2 = f'Images/{pred}/{pred}.jpg'

  #Check if file was successfully downloaded
  if os.path.isfile(file2): 
    img2=open(file2,'rb').read()
    wi2 = widgets.Image(value=img2, format='jpg', width=300, height=400)
    a=[wi1,wi2]
    wid=widgets.HBox(a)
    display.display(wid)


#get user input
while True:
  print('Enter a url to predict: (Enter "quit" to quit)\n')
  url = str(input())
  if url == "quit":
    break
  predict_by_url(url)